In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ELT_PySpark") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.2,com.amazonaws:aws-java-sdk-bundle:1.11.901") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "your username") \
    .config("spark.hadoop.fs.s3a.secret.key", "your password") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

25/05/23 13:05:55 WARN Utils: Your hostname, lenovo-PC resolves to a loopback address: 127.0.1.1; using 172.31.112.46 instead (on interface eth0)
25/05/23 13:05:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ubuntucoy/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntucoy/.ivy2/cache
The jars for the packages stored in: /home/ubuntucoy/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6bfc46d6-909f-4cb8-a1cb-41d66dec870e;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.1026 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 4204ms :: artifacts dl 59ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.1026 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.2 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.11.1026] in [default]
	-----------------------------------------------------

In [2]:
df = spark.read.csv("s3a://raw/crm/cust_info.csv", header=True, inferSchema=True)
df.show()

25/05/23 13:07:29 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
                                                                                

+------+----------+-------------+------------+------------------+--------+---------------+
|cst_id|   cst_key|cst_firstname|cst_lastname|cst_marital_status|cst_gndr|cst_create_date|
+------+----------+-------------+------------+------------------+--------+---------------+
| 11000|AW00011000|          Jon|       Yang |                 M|       M|     2025-10-06|
| 11001|AW00011001|       Eugene|     Huang  |                 S|       M|     2025-10-06|
| 11002|AW00011002|        Ruben|      Torres|                 M|       M|     2025-10-06|
| 11003|AW00011003|      Christy|         Zhu|                 S|       F|     2025-10-06|
| 11004|AW00011004|    Elizabeth|     Johnson|                 S|       F|     2025-10-06|
| 11005|AW00011005|        Julio|        Ruiz|                 S|       M|     2025-10-06|
| 11006|AW00011006|        Janet|     Alvarez|                 S|       F|     2025-10-06|
| 11007|AW00011007|        Marco|       Mehta|                 M|       M|     2025-10-06|

In [3]:
from pyspark.sql.functions import col, trim, upper, when, row_number
from pyspark.sql.window import Window

# Step 1: Window specification
window_spec = Window.partitionBy("cst_id").orderBy(col("cst_create_date").desc())

# Step 2: Filtering null first + apply transformation
df_transformed = (
    df.filter(col("cst_id").isNotNull())
      .select(
        "cst_id",
        "cst_key",
        trim(col("cst_firstname")).alias("cst_firstname"),
        trim(col("cst_lastname")).alias("cst_lastname"),
        when(upper(trim(col("cst_marital_status"))) == "S", "Single")
            .when(upper(trim(col("cst_marital_status"))) == "M", "Married")
            .otherwise("n/a")
            .alias("cst_marital_status"),
        when(upper(trim(col("cst_gndr"))) == "F", "Female")
            .when(upper(trim(col("cst_gndr"))) == "M", "Male")
            .otherwise("n/a")
            .alias("cst_gndr"),
        "cst_create_date"
      )
      .withColumn("flag_last", row_number().over(window_spec))  # create a column for the filter
      .filter(col("flag_last") == 1)  # filter
      .drop("flag_last")  # drop the column so it doesn't appear in the output
)

df_transformed.show()

[Stage 5:>                                                          (0 + 1) / 1]

+------+----------+-------------+------------+------------------+--------+---------------+
|cst_id|   cst_key|cst_firstname|cst_lastname|cst_marital_status|cst_gndr|cst_create_date|
+------+----------+-------------+------------+------------------+--------+---------------+
| 11000|AW00011000|          Jon|        Yang|           Married|    Male|     2025-10-06|
| 11001|AW00011001|       Eugene|       Huang|            Single|    Male|     2025-10-06|
| 11002|AW00011002|        Ruben|      Torres|           Married|    Male|     2025-10-06|
| 11003|AW00011003|      Christy|         Zhu|            Single|  Female|     2025-10-06|
| 11004|AW00011004|    Elizabeth|     Johnson|            Single|  Female|     2025-10-06|
| 11005|AW00011005|        Julio|        Ruiz|            Single|    Male|     2025-10-06|
| 11006|AW00011006|        Janet|     Alvarez|            Single|  Female|     2025-10-06|
| 11007|AW00011007|        Marco|       Mehta|           Married|    Male|     2025-10-06|

In [4]:
df_transformed.write \
    .mode("overwrite") \
    .option("header", True) \
    .csv("s3a://clean/crm/cust_info_clean.csv")

25/05/23 13:08:21 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/05/23 13:08:24 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
                                                                                

In [5]:
df_cleaned = spark.read.option("header", True).csv("s3a://clean/crm/cust_info_clean.csv")
df_cleaned.show()

+------+----------+-------------+------------+------------------+--------+---------------+
|cst_id|   cst_key|cst_firstname|cst_lastname|cst_marital_status|cst_gndr|cst_create_date|
+------+----------+-------------+------------+------------------+--------+---------------+
| 11000|AW00011000|          Jon|        Yang|           Married|    Male|     2025-10-06|
| 11001|AW00011001|       Eugene|       Huang|            Single|    Male|     2025-10-06|
| 11002|AW00011002|        Ruben|      Torres|           Married|    Male|     2025-10-06|
| 11003|AW00011003|      Christy|         Zhu|            Single|  Female|     2025-10-06|
| 11004|AW00011004|    Elizabeth|     Johnson|            Single|  Female|     2025-10-06|
| 11005|AW00011005|        Julio|        Ruiz|            Single|    Male|     2025-10-06|
| 11006|AW00011006|        Janet|     Alvarez|            Single|  Female|     2025-10-06|
| 11007|AW00011007|        Marco|       Mehta|           Married|    Male|     2025-10-06|

In [6]:
import duckdb

# Convert from Spark to Pandas
df_cleaned_cust_info = df_cleaned.toPandas()

# Connect to DuckDB
con = duckdb.connect("/mnt/d/data_engineering/duckdb/db/dev.duckdb")

# Save as DuckDB table (view)
con.register("crm_cust_info_clean_view", df_cleaned_cust_info)

# Save as DuckDB table (not view)
con.execute("CREATE OR REPLACE TABLE crm_cust_info_clean AS SELECT * FROM crm_cust_info_clean_view")

In [7]:
con.execute("SELECT * FROM crm_cust_info_clean LIMIT 10").df().style.hide(axis="index")

cst_id,cst_key,cst_firstname,cst_lastname,cst_marital_status,cst_gndr,cst_create_date
11000,AW00011000,Jon,Yang,Married,Male,2025-10-06
11001,AW00011001,Eugene,Huang,Single,Male,2025-10-06
11002,AW00011002,Ruben,Torres,Married,Male,2025-10-06
11003,AW00011003,Christy,Zhu,Single,Female,2025-10-06
11004,AW00011004,Elizabeth,Johnson,Single,Female,2025-10-06
11005,AW00011005,Julio,Ruiz,Single,Male,2025-10-06
11006,AW00011006,Janet,Alvarez,Single,Female,2025-10-06
11007,AW00011007,Marco,Mehta,Married,Male,2025-10-06
11008,AW00011008,Rob,Verhoff,Single,Female,2025-10-06
11009,AW00011009,Shannon,Carlson,Single,Male,2025-10-06


In [9]:
con.close()